In [20]:
import os
import numpy as np
import pandas as pd
import random

In [133]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
        
        
        params = {fundName:"sum" for fundName in self.getFundNames(funds)}
        params.update({"ID":lambda IDS: IDS.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in IDS])] }) #first id that is all characters or first id
        funds = funds.groupby("Ticker").agg(params)
        funds = funds.reset_index()

        #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return self.sp500["S&P500"].sum()/AddStocksPortfolio.Portfolio.sum()*100
    def addStocksPercentCEF(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return Portfolio.Portfolio.sum()/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        return self.simpleOverlap(weights)
        
        
        
        

In [139]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([longterm*0,1*current],axis=0)/1
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        percentCEF = self.addStocksPercentCEF(weights)/100
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return percentCEF * np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [140]:
class TabuSet:
    def inTabuSet(self,option):
        fund1, fund2 = option
        tabuList = [adjustment.getFunds() for adjustment in self.TabuSet[:self.numTabu]]
        return [fund1,fund2] in tabuList or [fund2,fund1] in tabuList

In [141]:
class Adjustment():
    def __init__(self,fund1,fund2,value,weights):
        self.fund1 = fund1
        self.fund2 = fund2
        self.value = value
        self.weights = weights
    def getValue(self):
        return self.value
    def getWeights(self):
        return self.weights
    def getFunds(self):
        return [self.fund1, self.fund2]

In [83]:
class runModel(AssetOverlap, Discount,TabuSet):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        self.numOptions = 3
        self.numTabu = 0
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        self.TabuSet = []
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(self.getFundWeight(fund2,weights),change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = weights.copy()
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def getFundWeight(self,fund,weights):
        return weights.loc[fund][0]
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)

        while self.EvaluateChange(fund1,fund2,weights,change)>control:
            #print(self.getFundWeight(fund2,weights), self.EvaluateChange(fund1,fund2,weights,change), control)
            self.changeWeights(fund1,fund2,weights,change)
            control = self.Evaluate(weights)
           
            
                
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        k = weights.copy()
        up = self.EvaluateChange(fund1,fund2,weights,change)
        
        down = self.EvaluateChange(fund2,fund1,weights,change)
        print(round(control,3),round(up,3), round(down,3))
        if down>control and down>up:
            fund1, fund2 = fund2, fund1
        elif control>=up: #if down>control but smaller than up that wouldn't make sense so...
            print(np.all(self.weights==k))
            return control,weights
        
        self.changeWeights(fund1,fund2,weights,change)
        #print("HERE", fund2, fund2, self.getFundWeight(fund2,weights))
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value, weights
    
    def randomFunds(self):
        fund2 = random.choice(self.weights.index)
        fund1 = random.choice(self.weights.index)
        
        if self.getFundWeight(fund1,self.weights)==0 and self.getFundWeight(fund2,self.weights)==0: 
            return self.randomFunds()
        return fund1, fund2
    
    def getOption(self,fund1,fund2):
        change = .1
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .01)
        return Adjustment(fund1=fund1,fund2=fund2,value=value,weights=weights)
    
            
    def runIteration(self):
        options = [self.randomFunds() for x in range(self.numOptions)]
        print(options)
        k = options[:]
        print(list(map(lambda funds: [self.getFundWeight(funds[0],self.weights),self.getFundWeight(funds[1],self.weights)], options)))
        options = list(filter(lambda option: not self.inTabuSet(option),options))
        
        options = [self.getOption(fund1,fund2) for fund1,fund2 in options]
        
        print([option.getValue() for option in options])
        bestIndx = np.argmax([option.getValue() for option in options])

        option = options[bestIndx]
        self.weights = option.getWeights()
        print(list(map(lambda funds: [self.getFundWeight(funds[0],self.weights),self.getFundWeight(funds[1],self.weights)], k)))
        self.TabuSet.append(option)
    
    def printEvaluation(self):
        print("Simple Overlap %.2f" % self.simpleOverlap(self.weights))
        print("Add Stocks Overlap %.2f" % self.addStocksOverlap(self.weights))
        print("Weighted Discount %.2f" % self.getWeightedDiscount(self.weights))
        
    def runModel(self):
        self.printEvaluation()
        
        for x in range(1000):
            self.runIteration()
            self.printEvaluation()
            print("\n\n")

In [84]:
obj = runModel()
obj.runModel()

Simple Overlap 31.15
Add Stocks Overlap 62.10
Weighted Discount -5.06
[('EOI', 'CEM'), ('EOI', 'SZC'), ('BDJ', 'BGY')]
[[0.020777569025012783, 0.0368625642182286], [0.020777569025012783, 0.03081201733686667], [0.013294582283052536, 0.0014604412716078797]]
28.522 28.983 28.04
30.209 30.507 29.871
31.042 31.107 30.594
[30.209152285917014, 31.04234378826721, 31.10680907706435]
[[0.08845215058010804, 0.0], [0.08845215058010804, 0.0], [0.014755023554660416, 0.0]]
Simple Overlap 36.35
Add Stocks Overlap 64.18
Weighted Discount -4.51



[('BUI', 'CET'), ('RNP', 'EOI'), ('ETJ', 'BCX')]
[[0.010344529899075484, 0.008325025555475226], [0.005387251212722673, 0.08845215058010804], [0.006549608613788282, 0.007052292539629828]]
31.107 30.597 31.719
31.74 31.363 31.941
31.941 32.037 31.804
[31.7396709954065, 31.940696291736252, 32.03740885686135]
[[0.0, 0.01866955545455071], [0.0, 0.0938394017928307], [0.01360190115341811, 0.0]]
Simple Overlap 37.55
Add Stocks Overlap 64.68
Weighted Discount -4.60





44.086 44.093 43.911
[44.01281481716251, 44.086430692309406, 44.093438359793765]
[[0.0, 0.011434583820250118], [0.0, 0.022594263411868137], [0.04455835629442049, 0.060358904211200166]]
Simple Overlap 51.31
Add Stocks Overlap 70.99
Weighted Discount -6.70



[('GRF', 'CLM'), ('GLQ', 'CPZ'), ('CHN', 'EOI')]
[[0.060358904211200166, 0.04017953034651409], [0.0, 0.024236176623597113], [0.040103538777778676, 0.0938394017928307]]
44.093 44.158 44.019
44.3 44.096 44.3
True
44.3 43.95 44.616
[44.299751064819944, 44.299751064819944, 45.52766132005645]
[[0.10053843455771425, 0.0], [0.0, 0.024236176623597113], [0.0, 0.13394294057060935]]
Simple Overlap 50.72
Add Stocks Overlap 70.69
Weighted Discount -7.67



[('CEM', 'EOS'), ('EXD', 'GRF'), ('BIF', 'AIO')]
[[0.03432844605291989, 0.0760591168733388], [0.0, 0.10053843455771425], [0.01800936859371189, 0.013614747477772549]]
45.528 45.277 45.765
46.306 46.088 46.306
True
46.306 46.456 46.12
[46.305829195702735, 46.305829195702735, 46.51033208740512]
[

Weighted Discount -7.70



[('BOE', 'BUI'), ('GRF', 'GLQ'), ('AOD', 'BOE')]
[[0.05166872665229988, 0.0], [0.08053843455771426, 0.0], [0.011434583820250118, 0.05166872665229988]]
51.169 51.169 50.85
True
51.169 51.169 51.045
True
51.169 51.17 51.136
[51.16894578374576, 51.16894578374576, 51.17034791659013]
[[0.04166872665229988, 0.0], [0.08053843455771426, 0.0], [0.021434583820250118, 0.04166872665229988]]
Simple Overlap 60.83
Add Stocks Overlap 76.14
Weighted Discount -7.71



[('MIE', 'STK'), ('CII', 'MIE'), ('BCX', 'GRF')]
[[0.0, 0.0360991142528612], [0.06753841666344375, 0.0], [0.01, 0.08053843455771426]]
51.17 51.13 51.17
True
51.17 51.17 51.061
True
51.17 51.033 51.244
[51.17034791659013, 51.17034791659013, 51.24414361779956]
[[0.0, 0.0360991142528612], [0.06753841666344375, 0.0], [0.0, 0.09053843455771425]]
Simple Overlap 60.77
Add Stocks Overlap 76.10
Weighted Discount -7.77



[('CII', 'CII'), ('ADX', 'BSTZ'), ('NFJ', 'AOD')]
[[0.06753841666344375, 0.06753841666344375], [0.1776

53.063 53.025 53.063
True
53.063 53.063 52.884
True
[53.06295168029338, 53.06295168029338, 53.06295168029338]
[[0.13073166985432824, 0.0], [0.0, 0.016099114252861195], [0.03, 0.0]]
Simple Overlap 62.87
Add Stocks Overlap 77.34
Weighted Discount -8.03



[('RNP', 'CII'), ('BOE', 'JEQ'), ('UTF', 'BIF')]
[[0.0, 0.047538416663443744], [0.03, 0.0], [0.0, 0.015496394114008072]]
53.063 52.956 53.063
True
53.063 53.063 52.812
True
53.063 52.734 53.063
True
[53.06295168029338, 53.06295168029338, 53.06295168029338]
[[0.0, 0.047538416663443744], [0.03, 0.0], [0.0, 0.015496394114008072]]
Simple Overlap 62.87
Add Stocks Overlap 77.34
Weighted Discount -8.03



[('GLQ', 'ADX'), ('SZC', 'AGD'), ('CHN', 'STK')]
[[0.0, 0.22931785150617784], [0.0, 0.03], [0.0, 0.016099114252861195]]
53.063 52.764 53.063
True
53.063 53.145 53.063
53.252 52.952 53.252
True
[53.06295168029338, 53.25161496965027, 53.25161496965027]
[[0.0, 0.22931785150617784], [0.03, 0.0], [0.0, 0.016099114252861195]]
Simple Overlap 62.66
A

Add Stocks Overlap 77.92
Weighted Discount -8.31



[('BME', 'STK'), ('BDJ', 'BDJ'), ('NFJ', 'JEQ')]
[[0.0, 0.016099114252861195], [0.053910580925194136, 0.053910580925194136], [0.13073166985432824, 0.0]]
53.736 53.564 53.736
True
53.736 53.736 53.736
True
53.736 53.736 53.521
True
[53.736399844447945, 53.736399844447945, 53.736399844447945]
[[0.0, 0.016099114252861195], [0.053910580925194136, 0.053910580925194136], [0.13073166985432824, 0.0]]
Simple Overlap 63.83
Add Stocks Overlap 77.92
Weighted Discount -8.31



[('EMO', 'EOS'), ('ASA', 'PEO'), ('RNP', 'BOE')]
[[0.0, 0.11038756292625868], [0.0, 0.04455835629442049], [0.0, 0.03]]
53.736 53.634 53.736
True
53.736 53.566 53.736
True
53.736 53.618 53.736
True
[53.736399844447945, 53.736399844447945, 53.736399844447945]
[[0.0, 0.11038756292625868], [0.0, 0.04455835629442049], [0.0, 0.03]]
Simple Overlap 63.83
Add Stocks Overlap 77.92
Weighted Discount -8.31



[('STK', 'CTR'), ('AWP', 'RQI'), ('SZC', 'AEF')]
[[0.016099114252861195, 0.0],

...

Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('BCX', 'EOS'), ('PEO', 'CEN'), ('GRF', 'CII')]
[[0.0, 0.04038756292625869], [0.04455835629442049, 0.0], [0.05053843455771427, 0.02753841666344374]]
56.126 55.979 56.126
True
56.126 56.126 56.101
True
56.126 56.088 56.101
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.04038756292625869], [0.04455835629442049, 0.0], [0.05053843455771427, 0.02753841666344374]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('EOS', 'NFJ'), ('SZC', 'AOD'), ('CPZ', 'PEO')]
[[0.04038756292625869, 0.0], [0.02, 0.0], [0.0, 0.04455835629442049]]
56.126 56.126 56.038
True
56.126 56.126 56.056
True
56.126 56.071 56.126
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.04038756292625869, 0.0], [0.02, 0.0], [0.0, 0.04455835629442049]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('SRV', 'EOI'), ('AIO', 'GRF'), ('BCX', 'BOE')]
[[0.0, 0.10

56.126 56.126 55.894
True
56.126 56.126 55.91
True
56.126 56.077 56.126
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.47018273278709594, 0.0], [0.04038756292625869, 0.0], [0.0, 0.02753841666344374]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('IGR', 'PEO'), ('BMEZ', 'PEO'), ('CLM', 'BOE')]
[[0.0, 0.04455835629442049], [0.0, 0.04455835629442049], [0.0, 0.019999999999999997]]
56.126 56.002 56.126
True
56.126 55.921 56.126
True
56.126 55.93 56.126
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.04455835629442049], [0.0, 0.04455835629442049], [0.0, 0.019999999999999997]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('STK', 'PEO'), ('BOE', 'BMEZ'), ('CPZ', 'BOE')]
[[0.0, 0.04455835629442049], [0.019999999999999997, 0.0], [0.0, 0.019999999999999997]]
56.126 56.073 56.126
True
56.126 56.126 55.92
True
56.126 56.066 56.126
True
[56.126167955684856, 56.126167955684856, 56.12616795568

Add Stocks Overlap 78.07
Weighted Discount -10.17



[('AGD', 'SZC'), ('AIO', 'BOE'), ('CET', 'RQI')]
[[0.0, 0.02], [0.0, 0.019999999999999997], [0.17683078410718947, 0.0]]
56.126 56.033 56.126
True
56.126 55.981 56.126
True
56.126 56.126 56.03
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.02], [0.0, 0.019999999999999997], [0.17683078410718947, 0.0]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('GRF', 'PEO'), ('EOI', 'CLM'), ('EOS', 'UTF')]
[[0.05053843455771427, 0.04455835629442049], [0.10605313173868339, 0.0], [0.04038756292625869, 0.0]]
56.126 56.08 56.081
True
56.126 56.126 55.931
True
56.126 56.126 55.749
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.05053843455771427, 0.04455835629442049], [0.10605313173868339, 0.0], [0.04038756292625869, 0.0]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('EOI', 'CII'), ('CRF', 'EOI'), ('ADX', 'ETJ')]
[[0.10605313173868339, 0.0275384

56.126 56.126 56.066
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.47018273278709594], [0.0, 0.02], [0.043910580925194134, 0.0]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('CHN', 'BDJ'), ('DSE', 'EOI'), ('SZC', 'IAF')]
[[0.0, 0.043910580925194134], [0.0, 0.10605313173868339], [0.02, 0.0]]
56.126 55.811 56.126
True
56.126 56.042 56.126
True
56.126 56.126 55.887
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.043910580925194134], [0.0, 0.10605313173868339], [0.02, 0.0]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('AOD', 'BDJ'), ('ADX', 'AEF'), ('CTR', 'EOI')]
[[0.0, 0.043910580925194134], [0.47018273278709594, 0.0], [0.0, 0.10605313173868339]]
56.126 56.05 56.126
True
56.126 56.126 55.888
True
56.126 55.966 56.126
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.043910580925194134], [0.47018273278709594, 0.0], [0.0, 0.10605313173868339]]
Simpl

56.126 55.987 56.126
True
56.126 55.877 56.126
True
56.126 55.987 56.126
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.02753841666344374], [0.0, 0.04038756292625869], [0.0, 0.04038756292625869]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17



[('IGR', 'CII'), ('ADX', 'BUI'), ('BOE', 'CHN')]
[[0.0, 0.02753841666344374], [0.47018273278709594, 0.0], [0.019999999999999997, 0.0]]
56.126 55.996 56.126
True
56.126 56.126 55.772
True
56.126 56.126 55.816
True
[56.126167955684856, 56.126167955684856, 56.126167955684856]
[[0.0, 0.02753841666344374], [0.47018273278709594, 0.0], [0.019999999999999997, 0.0]]
Simple Overlap 64.09
Add Stocks Overlap 78.07
Weighted Discount -10.17





In [88]:
list(obj.weights.index[obj.weights.weights!=0])

['ADX', 'SZC', 'CII', 'GRF', 'EOI', 'BDJ', 'EOS', 'CET', 'BOE', 'PEO']

In [85]:
obj.weights.sort_values(by="weights",ascending=False)

,weights
ADX,0.470183
CET,0.176831
EOI,0.106053
GRF,0.050538
PEO,0.044558
BDJ,0.043911
EOS,0.040388
CII,0.027538
SZC,0.020000
BOE,0.020000


In [147]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,longterm,value
Ticker,,,,,,
ADX,0.00,20.55,-13.27,-13.74,-4.304029,36.468780
AEF,7.90,2.64,-12.89,-14.09,-10.989011,33.155048
AGD,6.94,7.51,-12.70,-13.11,-3.754579,35.023824
AIO,2.95,5.05,-6.89,-10.25,-30.769231,9.423681
AOD,0.09,7.84,-13.81,-13.28,4.853480,41.063699
ASA,0.00,0.09,-15.96,-15.45,4.670330,47.143752
AWP,7.68,9.28,-12.22,-13.72,-13.736264,30.325558
BCX,1.20,6.63,-14.22,-15.29,-9.798535,37.350788
BDJ,0.27,7.19,-9.53,-9.34,1.739927,27.800770


In [114]:
def t(ab):
    ab = ab[:]
   
    ab.iloc[0] = [1,3]
    print(ab)
    
    
ki = pd.DataFrame([[1,2]])
t(ki)
ki
#i set the list copy to that and then though I could efdit it after word. but psych apparently that was me editing it and seeting list to something else doesn;t work othersie

   0  1
0  1  3


,0,1
0,1,3
